In [ ]:
import json
import pandas as pd
import pyodbc
import ast

# Function(s)

### Flatten JSON - Basement

In [ ]:
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

# Data I/O

In [ ]:
##### USED FOR TESTING
file1 = "C:/Users/R011846/Documents/data/experian_exp_full.txt"
s = open(file1, 'r').read()
data_exp_txt = ast.literal_eval(s) # raises an exception if the input isn't a valid Python datatype

In [ ]:
##### USED FOR TESTING
filename = "C:/Users/R011846/Documents/data/experian_full.json"
if filename:
    with open(filename, 'r') as f:
        data_tomra_json = json.load(f)

# Experian API Query - From Erica's Work

In [ ]:
# ADD EXPERIAN API QUERY

# SQL Stuff

### Create Connection & Cursor for Execution

In [ ]:
conn = pyodbc.connect('Driver={ODBC Driver 13 for SQL Server}; Server=tcp:dbdaz001.database.windows.net,1433; Database=IADevDB1;Uid=IAUSER1;Pwd=IADBUser1$; \
                      Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;')

In [ ]:
cursor = conn.cursor()

### Write JSON to Experian.CustomerData ADS Table

In [ ]:
cursor.execute("TRUNCATE TABLE [Experian].[CustomerData]")
# cursor.execute(
#     "SET IDENTITY_INSERT [Experian].[CustomerData] ON") # Set this to populate our own values for ID; off for 1up
conn.commit()

In [ ]:

# cursor.execute("INSERT INTO [Experian].[CustomerData](ID, JSON) values (?, ?)",int(1),str(data_exp_txt))
# cursor.execute("INSERT INTO [Experian].[CustomerData](ID, JSON) values (?, ?)",int(2),str(data_tomra_json))

cursor.execute("INSERT INTO [Experian].[CustomerData](JSON) values (?)",str(data_exp_txt))
cursor.execute("INSERT INTO [Experian].[CustomerData](JSON) values (?)",str(data_tomra_json))

conn.commit()

In [ ]:
cursor.close()

### Read JSON from Experian.CustomerData ADS Table

In [ ]:
sql_read = """SELECT *
         FROM [Experian].[CustomerData]
         WHERE [ID] = 1"""

In [ ]:
data_jsonSTR_experian = pd.read_sql(sql_read,conn)

In [ ]:
sql_read = """SELECT *
         FROM [Experian].[CustomerData]
         WHERE [ID] = 2"""

In [ ]:
data_jsonSTR_tomra = pd.read_sql(sql_read,conn)

### Close Connection

In [ ]:
conn.close()

# Work-Up

### Convert jsonSTR to Dictionary

In [ ]:
jsonDICT_exp = ast.literal_eval(data_jsonSTR_experian.JSON[0])
jsonDICT_tomra = ast.literal_eval(data_jsonSTR_tomra.JSON[0])
jsonDICT_tomra

In [ ]:
jsonDICT_tomra['results']['businessHeader']

### Flatten Dictionary for Pandas/CSV

In [ ]:
# Why 'earth'?  Because it's flat
earth_exp = flatten_json(jsonDICT_exp)
earth_tomra = flatten_json(jsonDICT_tomra)
print('Number of Fields for Experian JSON:',len(earth_exp))
print('Number of Fields for Tomra JSON:',len(earth_tomra))
earth_tomra

# Write Data to CSV

In [ ]:
# pd.DataFrame(
#     [earth_exp]).to_csv(
#     path_or_buf="C:/Users/R011846/Documents/Python/Scripts/sandbox/experian_json_parsing_EXPERIAN.csv",index=False)

In [ ]:
# pd.DataFrame(
#     [earth_tomra]).to_csv(
#     path_or_buf="C:/Users/R011846/Documents/Python/Scripts/sandbox/experian_json_parsing_TOMRA.csv",index=False)